In [1]:
import numpy as np
import pandas as pd
import sklearn

#Załadowanie danych

In [3]:
def fetch_financial_data(company='ALGN.US'):
    """
    This function fetches stock market quotations.
    """
    import pandas_datareader.data as web
    return web.DataReader(name=company, data_source='stooq')

df_raw = fetch_financial_data()
df_raw.head()

,Open,High,Low,Close,Volume
Date,,,,,
2022-08-26,271.96,271.96,252.12,252.45,868682
2022-08-25,265.64,273.24,265.61,272.99,569641
2022-08-24,252.68,262.92,251.60,261.70,703043
2022-08-23,255.63,258.30,251.16,252.23,601041
2022-08-22,263.14,266.82,255.08,256.05,648397


#Utworzenie kopii danych

In [4]:
df = df_raw.copy()
df = df[:5] #Wycięcie pięciu pierwszych wierszy
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2022-08-26 to 2022-08-22
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5 non-null      float64
 1   High    5 non-null      float64
 2   Low     5 non-null      float64
 3   Close   5 non-null      float64
 4   Volume  5 non-null      int64  
dtypes: float64(4), int64(1)
memory usage: 240.0 bytes


#Generowanie nowych zmiennych z daty

In [6]:
df.index.day

Int64Index([26, 25, 24, 23, 22], dtype='int64', name='Date')

In [8]:
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year
df

,Open,High,Low,Close,Volume,day,month,year
Date,,,,,,,,
2022-08-26,271.96,271.96,252.12,252.45,868682,26,8,2022
2022-08-25,265.64,273.24,265.61,272.99,569641,25,8,2022
2022-08-24,252.68,262.92,251.60,261.70,703043,24,8,2022
2022-08-23,255.63,258.30,251.16,252.23,601041,23,8,2022
2022-08-22,263.14,266.82,255.08,256.05,648397,22,8,2022


#Dyskretyzacja zmiennej ciągłej

In [9]:
df = pd.DataFrame(data={'height': [175., 178.5, 185., 191., 184.5, 183., 168.]})
df

,height
0,175.0
1,178.5
2,185.0
3,191.0
4,184.5
5,183.0
6,168.0


##Automatyczne tworzenie przedziałów

In [10]:
df['height_cat'] = pd.cut(x=df.height, bins=3)
df

,height,height_cat
0,175.0,"(167.977, 175.667]"
1,178.5,"(175.667, 183.333]"
2,185.0,"(183.333, 191.0]"
3,191.0,"(183.333, 191.0]"
4,184.5,"(183.333, 191.0]"
5,183.0,"(175.667, 183.333]"
6,168.0,"(167.977, 175.667]"


##Ręczne wyznaczanie przedziałów

In [12]:
df['height_cat'] = pd.cut(x=df.height, bins=(160, 175, 180, 195))
df

,height,height_cat
0,175.0,"(160, 175]"
1,178.5,"(175, 180]"
2,185.0,"(180, 195]"
3,191.0,"(180, 195]"
4,184.5,"(180, 195]"
5,183.0,"(180, 195]"
6,168.0,"(160, 175]"


##!!! Wyświetlanie wartości tekstowej zamiast przedziału

In [13]:
df['height_cat'] = pd.cut(x=df.height, bins=(160, 175, 180, 195), labels=['small', 'medium', 'high'])
df

,height,height_cat
0,175.0,small
1,178.5,medium
2,185.0,high
3,191.0,high
4,184.5,high
5,183.0,high
6,168.0,small


In [16]:
pd.get_dummies(df, drop_first=True, prefix='height')

,height,height_medium,height_high
0,175.0,0,0
1,178.5,1,0
2,185.0,0,1
3,191.0,0,1
4,184.5,0,1
5,183.0,0,1
6,168.0,0,0


#Ekstrakcja cech

In [17]:
df = pd.DataFrame(data={'lang': [['PL', 'ENG'], ['GER', 'ENG', 'PL', 'FRA'], ['RUS']]})
df

,lang
0,"[PL, ENG]"
1,"[GER, ENG, PL, FRA]"
2,[RUS]


##!!! Liczba elementów - funkcja apply(len)

In [18]:
df['lang_number'] = df['lang'].apply(len)
df

,lang,lang_number
0,"[PL, ENG]",2
1,"[GER, ENG, PL, FRA]",4
2,[RUS],1


## !!! Flaga jeśli wartość występuje

In [19]:
df['PL_flag'] = df['lang'].apply(lambda x: 1 if 'PL' in x else 0)
df

,lang,lang_number,PL_flag
0,"[PL, ENG]",2,1
1,"[GER, ENG, PL, FRA]",4,1
2,[RUS],1,0


#Dzielenie stringów na nowe kolumny

In [20]:
df = pd.DataFrame(data={'website': ['wp.pl', 'onet.pl', 'google.com']})
df

,website
0,wp.pl
1,onet.pl
2,google.com


In [23]:
# df['website'].str.split('.', expand = True)
df.website.str.split('.', expand = True)

,0,1
0,wp,pl
1,onet,pl
2,google,com


In [24]:
new = df['website'].str.split('.', expand = True)
df['portal'] = new[0]
df['ext.'] = new[1]
df

,website,portal,ext.
0,wp.pl,wp,pl
1,onet.pl,onet,pl
2,google.com,google,com
